#SAM Code for Vertical Swinging PV
---

Input data

## Importations and General Purpose Functions

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import calendar
import ast
import os


from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from scipy.integrate import quad
from math import isnan
from powermanagement import long_running

In [2]:
def extract_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            line = file.readline().strip()

            # Use ast.literal_eval to safely evaluate the contents and convert to a list
            extracted_list = ast.literal_eval(line)

            return extracted_list
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error occurred: {e}")
        return []

In [3]:
def create_time_frame(df):
    """
        converts the individual year, month, date, and hour of a
        dataframe into a datetime object in a column named Timeframe.

        returns a new dataframe
    """

    df = df.copy(deep=True)
    df = df.reset_index()
    df['Timeframe'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']],
                                     format = '%Y/%M/%D %H')
    return df

### Trigonometric Functions

In [4]:
def sinRad(x):
    return np.sin(x)

def cosRad(x):
    return np.cos(x)

def tanRad(x):
    return np.tan(x)

def sinDeg(x):
    return np.sin(x*pi/180)

def cosDeg(x):
    return np.cos(x*pi/180)

def tanDeg(x):
    return np.tan(x*pi/180)

def asinRad(x):
    return np.arcsin(x)

def acosRad(x):
    return np.arccos(x)

def atanRad(x):
    return np.arctan(x)

def asinDeg(x):
    return 180*np.arcsin(x)/pi

def acosDeg(x):
    return 180*np.arccos(x)/pi

def atanDeg(x):
    return 180*np.arctan(x)/pi

def cotanRad(x):
    return np.cos(x) / np.sin(x)

def cotanDeg(x):
    return np.cos(x*pi/180) / np.sin(x*pi/180)

def radToDeg(row):
    return row*180/(np.pi)

def increase_azim(x, azim_step):
    x = x + azim_step
    if x >= 360:
        x = x - 360
    return x

pi = np.pi

## Getting Input Data

### Weather Data

#### Extract Metadata from the Dataset

In [5]:
def extractWeatherData(url):

    def findDayOfYear(row):

        if row.year%4 != 0:
            return row.day_of_year

        else:
            if row.month == 1 or row.month == 2:
                return row.day_of_year
            else:
                return row.day_of_year - 1


    #____________________________Extract Metadata______________________________#
    df = pd.read_csv(url, nrows=1)
    metadata = df[['Latitude', 'Longitude', 'Time Zone', 'Elevation']]

    lat     = metadata.loc[0, 'Latitude']       # latitue
    lon     = metadata.loc[0, 'Longitude']      # longitude
    t_z     = metadata.loc[0, 'Time Zone']      # time zone
    elv     = metadata.loc[0, 'Elevation']      # elevation

    #___________________________Extract Timeseries_____________________________#
    df = pd.read_csv(url, skiprows=2)
    # create day of year and ensure February 29 is not included
    df = create_time_frame(df)
    df['Day of Year'] = df.Timeframe.apply(findDayOfYear)
    df = df.drop(df[(df.Month == 2) & (df.Day == 29)].index, axis=0)

    # time inputs
    year = df['Year']   # year
    mon = df['Month']   # month
    day = df['Day'] # day
    hour = df['Hour']   # hour
    min = df['Minute']  # minute
    doy = df['Day of Year'] # day of year

    # weather parameters inputs
    w_speed = df['Wind Speed']  # wind speed
    w_dir = df['Wind Direction']    # wind direction
    amb_temp = df['Temperature']    # ambient temperature
    albedo = df['Surface Albedo']   # surface albedo

    # irradation inputs
    beam_irr = df['DNI'] # beam irradiation (direct normal)
    diffuse_irr = df['DHI'] # diffuse horizontal irradiation
    global_irr = df['GHI']  # global horinzontal irradiation


    return (lat, lon, t_z, elv, year, mon, day, hour, min, doy, w_speed, w_dir,
            amb_temp, albedo, beam_irr, diffuse_irr, global_irr)

### Module Data

In [6]:
def getModuleParams(db_url, mod_name):
    df = pd.read_csv(db_url, skiprows=[1,2])
    df.set_index('Name', inplace=True)
    pv_module = df.loc[module_name]
    bifacial = pv_module.Bifacial
    pv_area = pv_module.A_c
    i_mp_ref = pv_module.I_mp_ref
    i_sc_ref = pv_module.I_sc_ref
    v_mp_ref = pv_module.V_mp_ref
    v_oc_ref = pv_module.V_oc_ref
    al_sc_ref = pv_module.alpha_sc
    be_oc_ref = pv_module.beta_oc
    i_l_ref = pv_module.I_L_ref
    i_o_ref = pv_module.I_o_ref
    rs_ref = pv_module.R_s
    rsh_ref = pv_module.R_sh_ref
    a_ref = pv_module.a_ref
    temp_adjust = pv_module.Adjust
    noct_temp = pv_module.T_NOCT

    return (bifacial, pv_area, i_mp_ref, i_sc_ref, v_mp_ref, v_oc_ref,
            al_sc_ref, be_oc_ref, i_l_ref, i_o_ref, rs_ref, rsh_ref, a_ref,
            temp_adjust, noct_temp)

### Inverter Data

In [7]:
def getInverterParams(db_url, inv_name):

    df = pd.read_csv(db_url, skiprows=[1,2])
    df.set_index('Name', inplace=True)

    inverter = df.loc[inverter_name]
    v_ac_nom = inverter.Vac
    v_dc_max = inverter.Vdcmax
    i_dc_max = inverter.Idcmax
    v_dc_nom = inverter.Vdco
    p_cons = inverter.Pso
    p_cons_nt = inverter.Pnt
    p_ac_max = inverter.Paco
    p_dc_max = inverter.Pdco
    v_mp_max = inverter.Mppt_high
    v_mp_min = inverter.Mppt_low
    inv_c0 = inverter.C0
    inv_c1 = inverter.C1
    inv_c2 = inverter.C2
    inv_c3 = inverter.C3

    return (v_ac_nom, v_dc_max, i_dc_max, v_dc_nom, p_cons, p_cons_nt, p_ac_max,
            p_dc_max, v_mp_max, v_mp_min, inv_c0, inv_c1, inv_c2, inv_c3)

## Sun Position Calculations

### Adjustment Functions

In [8]:
def adjustTimeOfDay(row):
    if row < 0:
        return row + 24
    elif row > 24:
        return row - 24
    else:
        return row


def adjustEclipticDeg(row):
    a = row - 360 * (row//360)
    if a >= 0:
        return a
    else:
        return a + 360


def adjustEclipticRad(row):
    a = row - 2*pi * (row//(2*pi))
    if a >= 0:
        return a
    else:
        return a + 2*pi


def adjustTime(row):
    a = row - 24 * (row//24)
    if a >= 0:
        return a
    else:
        return a + 24


def adjustRightAscen(col, eclong, obleq):
    for i in range(len(col)):
        if cosRad(eclong[i]) < 0:
            col[i] = col[i] + pi
        elif (cosRad(obleq[i]) * sinRad(eclong[i])) < 0:
              col[i] = col[i] + 2*pi
    return col


def adjustHourAngle(row):
    if row < -pi:
        return row + 2*pi
    elif row > pi:
        return row - 2*pi
    else:
        return row


def adjustAltAngleNoRefr(row):
    if -1 <= row <= 1:
        return asinRad(row)
    elif row > 1:
        return pi/2
    else:
        return -pi/2


def adjustAltAngleRefrStp1(row):
    if row > -0.56:
        res = (3.51561*(0.1594 + 0.0196*row +
                        0.00002*(row**2))) / (1 + 0.505*row + 0.0845*(row**2))
        return res
    else:
        return 0.56


def adjustAltAngleRefrStp2(row):
    if row > 90:
        return pi/2
    else:
        return (pi/180) * row


def adjustAzimuthAngle(row, alpha_0, HA):
    for i in range(len(row)):
        if -1 <= row[i] <= 1:
            row[i] = acosRad(row[i])
        elif (row[i] < -1) or (cosRad(alpha_0[i]) == 0):
            row[i] = pi
        elif row[i] > 1:
            row[i] = 0
        else:
            print("Nothing to do")

        if (-pi <= HA[i] <=0) or (HA[i] >= pi):
            row[i] = pi-row[i]
        elif 0 <= HA[i] <= pi:
            row[i] = pi + row[i]
    return row


def findSunUpDown(hour, h_rise, h_set):
    res = pd.Series(np.zeros(len(h_rise)))
    for i in range(len(h_rise)):
        if (hour[i] < h_rise[i]) or (hour[i] > h_set[i]):
            res[i] = 0
        else:
            res[i] = 1
    return res


def adjustEOT(row):
    if -0.33 <= row <= 0.33:
        return row
    elif row < -0.33:
        return row + 24
    elif row > 0.33:
        return row - 24

### Sun Position Angle Calculations Code

#### Find Sun Position Function

In [9]:
def findSunPositions(t_z, lat, lon, year, mon, doy, hour, min):

    #_______________________Effective Time Calculations________________________#
    ## calculate time of day as UTC offset
    tod = hour + min/60 - t_z
    ## corrected time of day
    tod = tod.apply(adjustTimeOfDay)
    ## julian date of the timestep
    julian = 32916.5 + 365*(year -
                            1949) + (year - 1949)/4 + doy + tod/24 - 51545


    #________________________________Sun Angles________________________________#
    ## mean longitude [deg] and asjustment
    mnlong = 280.46 + 0.9856474 * julian
    mnlong = mnlong.apply(adjustEclipticDeg)
    ## mean anomaly in [rad] and adjustment
    mnanom = (pi/180) * (357.528 + 0.9856003 * julian)
    mnanom = mnanom.apply(adjustEclipticRad)
    ## ecliptic longitude [rad] and asjustment
    eclong = (pi/180) * (mnlong + 1.915*sinRad(mnanom) + 0.02*sinRad(2*mnanom))
    eclong = eclong.apply(adjustEclipticRad)
    ## obliquity of ecliptic [rad]
    obleq = (pi/180) * (23.439 - 0.0000004*julian)

    ## celestial Coordinates
    # right ascension [rad] and correction
    ra = atanRad((cosRad(obleq) * sinRad(eclong)) / cosRad(eclong))
    ra = adjustRightAscen(ra, eclong, obleq)
    # declination angle [rad]
    declin = asinRad(sinRad(obleq) * sinRad(eclong))

    ## grenwhich mean siderial time [h]
    gmst = 6.697375 + 0.0657098242*julian + tod
    ## local mean siderial time [h] and adjustment
    lmst = gmst + lon/15
    lmst = lmst.apply(adjustTime)
    ## hour angle [rad]
    HA = 15*(pi/180)*lmst - ra
    HA = HA.apply(adjustHourAngle)
    # HA = HA.apply(adjustHourAngle)
    ## sun altitude angle [rad] (not corrected for refraction)
    alpha_0 = sinRad(declin) * sinRad(pi*lat/180) + \
                cosRad(declin) * cosRad(lat*pi/180) * cosRad(HA)
    alpha_0 = alpha_0.apply(adjustAltAngleNoRefr)
    ## sun altitude angle [rad] (corrected for refraction)
    alpha_0d = 180 * alpha_0 / pi
    r =  alpha_0d.apply(adjustAltAngleRefrStp1)
    alpha = alpha_0d + r
    alpha = alpha.apply(adjustAltAngleRefrStp2)
    ## sun azimuth angle [rad]
    gamma = (sinRad(alpha_0) * sinRad(lat*pi/180) - sinRad(declin)) / \
            (cosRad(alpha_0) * cosRad(lat*pi/180))
    gamma = adjustAzimuthAngle(gamma, alpha_0, HA)
    ## sun zenith angle [rad]
    sun_zenith = (pi/2) - alpha


    #________________________Sunrise and Sunset Hours__________________________#
    ## sunrise hour angle [rad]
    rise_HA = acosRad((cosDeg(90.833)/(cosDeg(lat)*cosRad(declin)) -
                    tanDeg(lat) * tanRad(declin)))
    ## equation of time [hours]
    EOT = ((180/pi)*ra - mnlong)/15
    EOT = (EOT.apply(adjustEOT))
    ## sunrise and sunset hour
    h_rise = 12 - (180/(15*pi))*rise_HA - (lon/15 - t_z) - EOT
    h_set = 12 + (180/(15*pi))*rise_HA - (lon/15 - t_z) - EOT
    ## sunposition up or down
    sunup = findSunUpDown(hour, h_rise, h_set)

    ## convert all angles in [deg]
    sun_zenith = sun_zenith.apply(radToDeg)
    alpha = alpha.apply(radToDeg)
    declin = declin.apply(radToDeg)
    gamma = gamma.apply(radToDeg)

    ### output results
    return sun_zenith, alpha, declin, gamma, sunup

### Extraterrestrial Radiation

In [10]:
def calcExtraTerrIrr(doy, sun_zen):
    extr_irr = 1367 * (1 + 0.033 * cosDeg(360*doy/365))
    conditions = [(sun_zen > 0) & (sun_zen < 90),
                sun_zen == 0,
                (sun_zen < 0) | (sun_zen >= 90)]
    choice_list = [extr_irr * cosDeg(sun_zen), extr_irr, 0]
    extr_irr = pd.Series(np.select(conditions, choice_list))

### True Solar Time and Eccentricity Correction
Provide Code if needed

## Surface Angles

In [11]:
def findSurfaceAngles(tilt, azim, sun_zen, sun_azim, sunup):
    #____________________________Tilt and Azimuth______________________________#
    # both angles in [deg]
    surf_tilt = tilt
    surf_azim = azim

    #____________________________Angle of Incidence____________________________#
    aoi_temp = sinDeg(sun_zen) * cosDeg(sun_azim - surf_azim) * \
                sinDeg(surf_tilt) + cosDeg(sun_zen) * cosDeg(surf_tilt)

    conditions = [aoi_temp < -1,
                  aoi_temp > 1,
                  (aoi_temp >= -1) & (aoi_temp <= 1)]
    choice_list = [pi, 0, acosRad(aoi_temp)]
    aoi_temp = pd.Series(np.select(conditions, choice_list))
    aoi = aoi_temp.apply(radToDeg)

    ## make angle of incidence 0 during nighttime
    conditions = [sunup == 0,
                  sunup == 1]
    choice_list = [0, aoi]
    aoi = pd.Series(np.select(conditions, choice_list))

    return aoi, surf_tilt, surf_azim

## Plan of Array (POA) Irradiance

### Helper Functions

In [12]:
def adjust_POA(row):
    if row < 0:
        return 0
    else:
        return row

### Main Function

In [13]:
def findPoaIrradiances(glob_irr, beam_irr, diff_irr, extr_irr, alb, aoi,
                       surf_tilt, sun_zen, irr_mode=0, sky_model=0, use_alb=1):

    #___________________________POA Beam Irradiance____________________________#
    if irr_mode == 0:
        poa_beam = beam_irr * cosDeg(aoi)
    elif irr_mode == 2:
        pass
    poa_beam = poa_beam.apply(adjust_POA)


    ## beam irradiance on horizontal surface
    poa_beam_hor = beam_irr * cosDeg(sun_zen)
    if (poa_beam_hor > extr_irr).any == True:
        res = 'Error: Horizontal Beam Irradiation Greater Than' + \
              ' Extraterrestrial Irradiation'
        return res


    #________________________POA Sky Diffuse Irradiance________________________#
    if irr_mode == 0:
        diff_irr = diff_irr
    elif irr_mode == 2:
        pass
    else:
        pass

    if sky_model == 0:
        poa_diff = diff_irr * (( 1 + cosDeg(surf_tilt))/2)
    elif sky_model == 1:
        pass
    elif sky_model == 2:
        pass
    else:
        pass


    #_________________________POA Reflected Irradiance_________________________#
    poa_refl = alb * (beam_irr * cosDeg(sun_zen) +
                      diff_irr) * ((1 - cosDeg(surf_tilt))/2)

    return poa_beam, poa_diff, poa_refl

## Effective POA Irradiance

In [14]:
def findEffectivePoa(poa_beam, poa_diff, poa_refl, soiling=5, beam_timestep=0,
                     beam_loss=0, diff_loss=0):
    #_________________________Nominal POA Irradinace___________________________#
    nom_irr = poa_beam + poa_diff + poa_refl

    #_______________________Soiling and Shading Factors________________________#
    theta_soiling = 1 - soiling/100
    theta_beam = 1 - beam_loss/100
    theta_diff = 1 - diff_loss/100

    #___________________Irradiance After Shading And Losses____________________#
    poa_beam_eff = poa_beam * theta_soiling * theta_beam * theta_diff
    poa_diff_eff = poa_diff * theta_soiling * theta_beam * theta_diff
    poa_refl_eff = poa_refl * theta_soiling * theta_beam * theta_diff
    poa_tot = poa_beam_eff + poa_diff_eff + poa_refl_eff
    poa_tot_diff = poa_diff_eff + poa_refl_eff

    return (nom_irr, poa_beam_eff, poa_diff_eff, poa_refl_eff,
            poa_tot, poa_tot_diff)

## Module DC Output

### Helper Functions

#### NOCT Temperature Model

In [15]:
def findNoctTemp(pv_irr, tau_al, w_speed, air_temp, noct_temp, imp, vmp,
                 pv_area, pv_height=2, standoff=5):

    ## reference Efficiency
    eta_ref = imp * vmp / (pv_area * 1000)
    ## windspeed adjustment
    if pv_height < 3:
        w_speed_adj = 0.51 * w_speed
    else:
        w_speed_adj = 0.61 * w_speed
    ## noct temperature adjustment
    if 6.35 < standoff <= 8.89:
        noct_temp_adj = noct_temp + 2
    elif 3.81 < standoff <= 6.35:
        noct_temp_adj = noct_temp + 6
    elif 1.27 < standoff <= 3.81:
        noct_temp_adj = noct_temp + 11
    elif standoff <= 1.27:
        noct_temp_adj = noct_temp + 18
    else:
        noct_temp_adj = noct_temp
    ## cell temperature
    cell_temp = air_temp + (pv_irr/800) * (noct_temp_adj - 20) * \
                (1 - eta_ref/tau_al) * 9.5/(5.7 + 3.8*w_speed_adj)
    ## return result
    return cell_temp

#### Maximum Power Voltage and Current Calculations

In [16]:
def findMaxPowerParamsSingle(r_sh, a, i_l, i_o, r_s, vref, iref):
    """
        Optimization function that finds the mpp voltage and mpp current for
        a given operating condition

    """


    def my_func(x):
        x1 = x[0]
        x2 = x[1]
        d1 = (x1 + x2*r_s) / a
        d2 = (x1 + x2*r_s) / r_sh
        g = i_l - i_o*(np.exp(d1) - 1) - d2 - x2
        f = ((i_o/a)*(np.exp(d1)) + 1/r_sh) / (1 + r_s/r_sh +
                                               (i_o*r_s/a)*np.exp(d1)) - x2/x1
        return [f, g]


    x0 = np.array([1, 1])

    res = fsolve(my_func, x0, xtol=1e-4)

    return res


def findMaxPowerParams(r_sh, a, i_l, i_o, r_s, vref, iref, cell_temp):
    v_mp = pd.Series(np.zeros(len(i_l)))
    i_mp = pd.Series(np.zeros(len(i_l)))

    for m in range(len(cell_temp)):
        if isnan(cell_temp[m]):
            v_mp[m] = 0
            i_mp[m] = 0
            continue
        rsh_m = r_sh[m]
        a_m = a[m]
        il_m = i_l[m]
        io_m = i_o[m]
        rs_m = r_s
        vref_m = vref
        iref_m = iref
        vi_mp = findMaxPowerParamsSingle(rsh_m, a_m, il_m, io_m, rs_m, vref_m,
                                         iref_m)
        v_mp[m] = vi_mp[0]
        i_mp[m] = vi_mp[1]

    return v_mp, i_mp

#### POA Front After Reflection

In [17]:
def poaAfterReflection(poa_beam, poa_diff, poa_refl, aoi, surf_tilt, sunup):

    #___________________Transmittance as a Function of Theta___________________#
    def tau_alpha_calc(theta_i):
        # refraction angle
        theta_r = asinDeg(sinDeg(theta_i) / n)

        theta_num = theta_r - theta_i
        theta_den = theta_r + theta_i
        tau_a = np.exp(-(K*L)/cosDeg(theta_r))
        sin_val = ((sinDeg(theta_num))**2) / ((sinDeg(theta_den))**2)
        tan_val = ((tanDeg(theta_num))**2) / ((tanDeg(theta_den))**2)
        tau_t = 1 - 0.5*(sin_val + tan_val)
        res = tau_a * tau_t
        return res


    #___________________________Constant Values _______________________________#
    n = 1.526; L = 0.002; K = 4
    #______________________Global Effective Irradiance_________________________#
    ##################################################
    #glob_irr = poa_beam + poa_diff + poa_refl #######
    ##################################################
    #______________________________Incide Angles_______________________________#
    # sky-diffuse incident angle
    theta_d = 59.7 - 0.1388*surf_tilt + 0.001497*np.square(surf_tilt)
    # ground-reflected incident angle
    theta_g = 90.0 - 0.5788*surf_tilt + 0.002693*np.square(surf_tilt)
    #____________________Transmittance and Angle Modifiers_____________________#
    ## normal surface
    tau_al_n = np.exp(-K*L)*(1 - ((1-n)/(n+1))**2)
    ## beam irradiance
    tau_al_b = tau_alpha_calc(aoi)
    conditions = [sunup == 0,
                  sunup == 1]
    choice_list = [0, tau_al_b]
    tau_al_b = pd.Series(np.select(conditions, choice_list))
    k_tau_b = tau_al_b / tau_al_n
    ## sky diffuse
    tau_al_d = tau_alpha_calc(theta_d)
    k_tau_d = tau_al_d / tau_al_n
    ## ground diffuse
    tau_al_g = tau_alpha_calc(theta_g)
    k_tau_g = tau_al_g / tau_al_n
    ## irradiance absorbed by pv cell
    pv_irr = poa_beam * k_tau_b + poa_diff * k_tau_d + \
                poa_refl * k_tau_g

    return pv_irr

### CEC Module Model

In [18]:
def runCecModuleModel(poa_beam_eff, poa_diff_eff, poa_refl_eff, poa_beam_rear,
                      poa_diff_rear, poa_refl_rear, aoi, sun_zen, sunup,
                      amb_temp, noct_temp, w_speed, elv, hod, surf_tilt,
                      i_mp_ref, v_mp_ref, v_oc_ref, i_sc_ref, al_sc_ref,
                      be_oc_ref, i_l_ref, i_o_ref, rs_ref, a_ref, rsh_ref,
                      temp_adjust, pv_area, pv_height=2, standoff=5):


    #________________________Zenith Adjustment Function________________________#
    def adjustZenith(row):
        if row < 0:
            return 0
        elif row > 86:
            return 86
        else:
            return row


    pv_irr_front = poaAfterReflection(poa_beam_eff, poa_diff_eff, poa_refl_eff,
                                      aoi, surf_tilt, sunup)

    ###########################################################################
    (poa_beam_rear, poa_diff_rear, poa_refl_rear) = (0, 0, 0)
    pv_irr_rear = 0
    ###########################################################################

    pv_irr = pv_irr_front + pv_irr_rear
    ## Global Effective Irradiance
    glob_irr = poa_beam_eff + poa_diff_eff + poa_refl_eff + poa_beam_rear + \
                poa_diff_rear + poa_refl_rear
    ## transmittance absorptance product
    tau_alpha = 0.9 * pv_irr / glob_irr
    conditions = [tau_alpha.isna(),
                    tau_alpha.isna() == 0]
    choice_list = [0, tau_alpha]
    tau_alpha = pd.Series(np.select(conditions, choice_list))

    #___________________________Air Mass Calculation___________________________#
    ## constrain zenith angle
    sun_zen = sun_zen.apply(adjustZenith)
    ## air mass with sea level correction factor
    air_mass = np.exp(-0.0001184*elv) * \
                (cosDeg(sun_zen) + 0.5057*((96.080 - sun_zen)**(-1.634)))**(-1)
    ## air mass modifiers
    a0 = 0.918093; a1 = 0.086257; a2 = -0.024459; a3 = 0.002816; a4 = -0.000126
    ## modified air mass
    air_mass_mod = a0 + a1 * air_mass + a2 * air_mass**2 + a3 * air_mass**3 + \
                    a4 * air_mass**4
    ## adjusted effective transmitted irradiance
    pv_irr_adj = air_mass_mod * pv_irr
            # return air_mass_mod
    if (pv_irr_adj > 1).any == False:
        res = 'Error: Wrong Value of Effective Transmitted Irradiance'
        return res
    #___________________________PV Module Calculations_________________________#
    mu_i_sc = al_sc_ref * (1 - temp_adjust/100)
    be_v_oc = be_oc_ref * (1 + temp_adjust/100)
    ## cell temperature
    cell_temp = findNoctTemp(pv_irr_adj, tau_alpha, w_speed, amb_temp,
                             noct_temp, i_mp_ref, v_mp_ref, pv_area,
                             pv_height=2, standoff=5)
    conditions = [cell_temp.isna(),
                cell_temp.isna() == 0]
    choice_list = [0, cell_temp]
    cell_temp_adj = pd.Series(np.select(conditions, choice_list))
    cell_temp_K = cell_temp_adj + 273.15
    ## reference temperature
    ref_temp = 25 + 273.15
    ## light current
    i_l = (pv_irr_adj/1000) * (i_l_ref + mu_i_sc * (cell_temp_K - ref_temp))
    ## diode reverse saturation current
    # boltzman constant
    k = 8.618*10**-5
    # cell bandgap
    e_gap = 1.12 * (1 - 0.0002677*(cell_temp_K - ref_temp))
    # reverse current
    i_o = i_o_ref * (cell_temp_K/ref_temp)**3 * np.exp((1/k) *
                                                       (1.12/ref_temp -
                                                        e_gap/cell_temp_K))

    ## single diode module current at Voc (I=0)

    ## ideality factor
    a = a_ref * cell_temp_K / ref_temp
    ## shunt resistance
    r_sh = rsh_ref * 1000 / pv_irr_adj
    ## short circuit current

    ## series resistance
    r_s = rs_ref
    i_l = i_l

    ## get maximum point values of voltage and current
    v_mp, i_mp = findMaxPowerParams(r_sh, a, i_l, i_o, r_s, v_oc_ref,
                                    i_sc_ref, cell_temp)
    eta_m = i_mp * v_mp / (pv_area * 1000)
    p_mp = i_mp * v_mp
    return v_mp, i_mp, eta_m, p_mp, pv_irr_adj, cell_temp_adj

## Array DC Output

In [19]:
def findOneArrayOutput(v_mp, p_mp, mism_loss=2, diode_conn_loss=0.5,
                       dc_wiring_loss=2, track_loss=0, name_plate_loss=0,
                       dc_optim_loss=0, snow_loss=0, mod_per_str=6,
                       parallel_str=3):

    #____________________________Array Calculations____________________________#
    ## array dc voltage
    v_dc = v_mp * mod_per_str
    ## array oc voltage and sc current

    ## array dc loss factors calculation
    f_dc1 = 1 - mism_loss/100
    f_dc2 = 1 - diode_conn_loss/100
    f_dc3 = 1 - dc_wiring_loss/100
    f_dc4 = 1 - track_loss/100
    f_dc5 = 1 - name_plate_loss/100
    f_dc6 = 1 - dc_optim_loss/100
    f_dc = f_dc1 * f_dc2 * f_dc3 * f_dc4 * f_dc5 * f_dc6
    ## snow loss factor calculation
    f_snow = 1 - snow_loss/100
    ## subarray mismatch losses

    ## array power output
    p_dc = mod_per_str * parallel_str * p_mp * f_dc * f_snow
    return v_dc, p_dc

## Inverter AC Output

In [20]:
def runCecInverterModel(p_dc, v_dc, v_ac_nom, v_dc_max, i_dc_max, v_dc_nom,
                        p_cons, p_cons_nt, p_ac_max, p_dc_max, v_mp_max,
                        v_mp_min, inv_c0, inv_c1, inv_c2, inv_c3, datasheet=0,
                        lossless=0, clip_en=1, night_cons=1):

    #____________________________SANDIA Model CEC______________________________#
    ## model parameters
    if datasheet == 1:
        inv_c0 = 0
        inv_c1 = 0
        inv_c2 = 0
        inv_c3 = 0
    A = p_dc_max * ( 1 + inv_c1 * (v_dc - v_dc_nom))
    B = p_cons * ( 1 + inv_c2 * (v_dc - v_dc_nom))
    if lossless:
        B = 0
    C = inv_c0 * (1 + inv_c3 * (v_dc - v_dc_nom))

    ## model equation
    p_ac = ((p_ac_max/(A-B)) - C*(A-B)) * (p_dc - B) + C * (p_dc - B)**2
    ## operating power losses
    B = 0
    p_ac_no_loss = ((p_ac_max/(A-B)) - C*(A-B)) * (p_dc - B) + C * (p_dc - B)**2
    p_ac_loss = p_ac_no_loss - p_ac
    ## adjust ac power for night operation consumption
    p_ac_adj = p_ac * 1
    if night_cons:
        conditions = [p_dc < p_cons,
                    p_dc >= p_cons]
        choice_list = [-p_cons_nt, p_ac]
        p_ac_adj = pd.Series(np.select(conditions, choice_list))
    ## clipping losses
    p_ac_no_clip = p_ac_adj * 1
    if clip_en:
        conditions = [p_ac_adj > p_ac_max,
                    p_ac_adj <= p_ac_max]
        choice_list = [p_ac_max, p_ac_adj]
        p_ac_adj = pd.Series(np.select(conditions, choice_list))
    p_ac_clip = p_ac_no_clip - p_ac_adj
    ## inverter efficiency and adjustment
    eta_inv = p_ac_adj / p_dc
    conditions = [(eta_inv.isna()) | (eta_inv < 0),
                eta_inv.isna() == 0]
    choice_list = [0, eta_inv]
    eta_inv = pd.Series(np.select(conditions, choice_list))

    return p_ac_adj, eta_inv, p_ac_clip, p_ac_loss

## Year 1 Analysis of Output Power

In [21]:
def outputYearOne(mon, day, hour, doy, p_ac, p_dc):
    year_1_power = pd.DataFrame(mon)
    year_1_power['Day'] = day
    year_1_power['Hour'] = hour
    year_1_power['day_of_year'] = doy
    year_1_power['Power_DC'] = p_dc
    annual_dc_energy = p_dc.sum()/1000
    annual_dc_energy = p_dc.sum()/1000
    monthly_dc_energy = year_1_power.groupby('Month')['Power_DC'].sum()/1000
    # monthly_dc_energy = monthly_dc_energy.to_frame()
    daily_dc_energy = year_1_power.groupby('day_of_year')['Power_DC'].sum()/1000

    return (annual_dc_energy, monthly_dc_energy,
            daily_dc_energy, annual_dc_energy)

## Lifetime Analysis of output Power

In [22]:
def outputLifetime(lifetime, degr_rate, annual_dc_energy):

    lifetime_dc_energy = annual_dc_energy
    annual_dc_energy_y = annual_dc_energy
    for i in range(lifetime):
        annual_dc_energy_y = annual_dc_energy_y * (1 - degr_rate)
        lifetime_dc_energy += annual_dc_energy_y

    return lifetime_dc_energy

## Simulation

In [23]:
def runSimulation(weather_url, module_name, inverter_name, tilt, azimuth,
                  mod_wgt, g, air_density, h_high, lifetime, degr_rate_percent,
                  mism_loss, diode_conn_loss, dc_wiring_loss, track_loss,
                  name_plate_loss, dc_optim_loss, snow_loss, mod_per_str,
                  parallel_str):

    ## extract weather data
    res = extractWeatherData(weather_url)
    (lat, lon, t_z, elv, year, mon, day, hour, min, doy, w_speed, w_dir,
     amb_temp, albedo, beam_irr, diffuse_irr, global_irr) = res

    ## get modules parameters
    res = getModuleParams(module_database_url, module_name)
    (bifacial, pv_area, i_mp_ref, i_sc_ref, v_mp_ref, v_oc_ref, al_sc_ref,
    be_oc_ref, i_l_ref, i_o_ref, rs_ref, rsh_ref, a_ref, temp_adjust,
    noct_temp) = res

    ## get inverter parameters
    res = getInverterParams(inverter_database_url, inverter_name)
    (v_ac_nom, v_dc_max, i_dc_max, v_dc_nom, p_cons, p_cons_nt, p_ac_max,
     p_dc_max, v_mp_max, v_mp_min, inv_c0, inv_c1, inv_c2, inv_c3) = res

    ## sun angles
    (sun_zen, sun_alt, sun_decl, sun_azim,
     sunup) = findSunPositions(t_z, lat, lon, year, mon, doy, hour, min)

    ## extraterrestrial irradiation
    extr_irr = calcExtraTerrIrr(doy, sun_zen)

    ## surface angles
    aoi, surf_tilt, surf_azim = findSurfaceAngles(tilt, azimuth, sun_zen,
                                                  sun_azim, sunup)

    ## poa front irradiances
    (poa_beam, poa_diffuse,
     poa_reflected) = findPoaIrradiances(global_irr, beam_irr, diffuse_irr,
                                         extr_irr, albedo, aoi, surf_tilt,
                                         sun_zen)

    (poa_beam_rear, poa_diff_rear, poa_refl_rear) = (0, 0, 0)

    ## effective poa irradiances front and back
    res = findEffectivePoa(poa_beam, poa_diffuse, poa_reflected, soiling=5,
                            beam_timestep=0, beam_loss=0, diff_loss=0)
    (nom_irr, poa_beam_eff, poa_diff_eff, poa_refl_eff, poa_tot,
     poa_tot_diff) = res

    ## run module model to get operation parameters
    res = runCecModuleModel(poa_beam_eff, poa_diff_eff, poa_refl_eff,
                            poa_beam_rear, poa_diff_rear, poa_refl_rear, aoi,
                            sun_zen, sunup, amb_temp, noct_temp, w_speed, elv,
                            hour, surf_tilt, i_mp_ref, v_mp_ref, v_oc_ref,
                            i_sc_ref, al_sc_ref, be_oc_ref, i_l_ref, i_o_ref,
                            rs_ref, a_ref, rsh_ref, temp_adjust, pv_area,
                            pv_height=3.65, standoff=9)
    v_mp, i_mp, eta_m, p_mp, pv_irr_adj, cell_temp_adj = res

    ## find array parameters
    v_dc, p_dc = findOneArrayOutput(v_mp, p_mp, mism_loss, diode_conn_loss,
                                    dc_wiring_loss, track_loss, name_plate_loss,
                                    dc_optim_loss, snow_loss, mod_per_str,
                                    parallel_str)

    ## run inverter model to get ac output
    res = runCecInverterModel(p_dc, v_dc, v_ac_nom, v_dc_max, i_dc_max,
                              v_dc_nom, p_cons, p_cons_nt, p_ac_max, p_dc_max,
                              v_mp_max, v_mp_min, inv_c0, inv_c1, inv_c2,
                              inv_c3, datasheet=0, lossless=0, clip_en=1,
                              night_cons=1)
    p_ac, eta_inv, p_ac_clip, p_ac_loss = res

    ## annual output
    (annual_dc_energy, monthly_dc_energy,
     daily_dc_energy, annual_dc_energy) = outputYearOne(mon, day, hour, doy,
                                                        p_ac, p_dc)

    ## lifetime output
    lifetime = lifetime + 1
    degr_rate = degr_rate_percent/100
    lifetime_dc_energy = outputLifetime(lifetime, degr_rate, annual_dc_energy)

    return (annual_dc_energy, monthly_dc_energy, daily_dc_energy,
            lifetime_dc_energy, surf_tilt, v_mp)

In [24]:
module_database_url = 'CEC_Modules.csv'
inverter_database_url = 'CEC_Inverters.csv'

module_name = 'Heliene 144HC-470'
inverter_name = 'SolarEdge Technologies Ltd : SE6000x [208V]'

city_name = 'London_Ontario'
# weather_url = 'london_ontario_42.984267_-81.247534_psm3-tmy_60_tmy.csv'
weather_url_dict = {
    'bunny': ['Stanford', 'stanford_37.4314_-122.169_psm3-tmy_60_tmy.csv', 30],
    'pyramid': ['Cairo', 'cairo_msg-iodc_60_2019.csv', 26],
    'rhodes': ['Athens', 'athens_msg-iodc_60_2019.csv', 30],
    'samothrace': ['Athens', 'athens_msg-iodc_60_2019.csv', 30],
    'thinker-extremely-low': ['Paris', 'paris_msg-iodc_60_2019.csv', 30], 
    'thinker-low': ['Paris', 'paris_msg-iodc_60_2019.csv', 30],
    'thinker-med': ['Paris', 'paris_msg-iodc_60_2019.csv', 30],
    'tree-top': ['Lome', 'lome_msg-iodc_60_2019.csv', 10],
    'inukshuk': ['London', 'london_ontario_42.984267_-81.247534_psm3-tmy_60_tmy.csv', 34],
    'inukshuk_toronto': ['Toronto', 'toronto_43.6519_-79.3817_psm3-tmy_60_tmy.csv', 35]
                   }                             

In [25]:
building_model_list = []
while True:
    my_file_name = input("Enter File Name or Enter 0 to Exit:...")
    
    if(my_file_name == '0'):
        break
        print("Exited loop successfully")
    elif(os.path.exists(my_file_name+'-sam-input.txt')):
        building_model_list.append(my_file_name+'-sam-input.txt')
        print(my_file_name + " has been added to the model list")
    else:
        print(my_file_name + " is not a valid file name, please try again")
    

building_model_list


Enter File Name or Enter 0 to Exit:...inukshuk
inukshuk has been added to the model list
Enter File Name or Enter 0 to Exit:...inukshuk_toronto
inukshuk_toronto is not a valid file name, please try again
Enter File Name or Enter 0 to Exit:...inukshuk_toronto
inukshuk_toronto has been added to the model list
Enter File Name or Enter 0 to Exit:...0


['inukshuk-sam-input.txt', 'inukshuk_toronto-sam-input.txt']

In [26]:
@long_running

def my_long_sim_code():
    azim_step = 10


    writer1 = pd.ExcelWriter('azimuth_opt_results.xlsx')
    writer2 = pd.ExcelWriter('annual_results.xlsx')

    
    for file_path in building_model_list:
        
        sim_start = time.time()
        
        file_path_short = file_path[0:-14]
        weather_url = weather_url_dict[file_path_short][1]
        building_location = weather_url_dict[file_path_short][0]

        print('---------------Simulating ' + file_path[0:-4] + ' in ' + building_location + '--------------')
        # file_path = "output.txt"
        my_data_list = extract_list_from_file(file_path)

        my_df = pd.DataFrame(my_data_list,
                            columns=['tilt_angles',
                                    'azim_angles',
                                    'num_cells'
                                    ]
                            )
        my_df.tilt_angles = my_df.tilt_angles.apply(radToDeg)
        my_df.azim_angles = my_df.azim_angles.apply(radToDeg)

        my_dict = my_df.transpose().to_dict('list')
        new_keys = ["Face "+ str(x) for x in my_dict.keys()]
        my_params_list = list(my_dict.values())
        my_building_faces = dict(zip(new_keys, my_params_list))

        annual_dc_energy_dic = {}
        monthly_dc_energy_dic = {}
        daily_dc_energy_dic = {}


        for a_face in my_building_faces:
            tilt = my_building_faces[a_face][0]
            azimuth = my_building_faces[a_face][1]
            num_modules = my_building_faces[a_face][2]
            ############################################################################
            mod_wgt = 20
            g = 10
            air_density = 1.225
            h_high = 1.04+0.15
            ############################################################################

            lifetime = 26
            degr_rate_percent = 0.5

            #________________________________DC Losses_________________________________#
            ## mismatch losses in percentage
            mism_loss = 2
            diode_conn_loss = 0.5
            dc_wiring_loss = 2
            track_loss = 0
            name_plate_loss = 0
            dc_optim_loss = 0
            snow_loss = 0
            #_____________________________Array Parameters_____________________________#
            ## modules per string
            mod_per_str = 1
            parallel_str = 1
            res = runSimulation(weather_url, module_name, inverter_name, tilt, azimuth,
                            mod_wgt, g, air_density, h_high, lifetime, degr_rate_percent,
                            mism_loss, diode_conn_loss, dc_wiring_loss, track_loss,
                            name_plate_loss, dc_optim_loss, snow_loss, mod_per_str,
                            parallel_str)

            (annual_dc_energy, monthly_dc_energy, daily_dc_energy,
            lifetime_dc_energy, surf_tilt, v_mp) = res

            annual_dc_energy_dic[a_face] = [annual_dc_energy*num_modules]
            monthly_dc_energy_dic[a_face] = [monthly_dc_energy*num_modules]
            daily_dc_energy_dic[a_face]= [daily_dc_energy*num_modules]

        annual_dc_energy_df = pd.DataFrame.from_dict(annual_dc_energy_dic)
        monthly_dc_energy_df = pd.DataFrame.from_dict(monthly_dc_energy_dic)
        daily_dc_energy_df = pd.DataFrame.from_dict(daily_dc_energy_dic)


        #----------------------OPTIMIZED TILT-----------------------------------#
        tilt = weather_url_dict[file_path_short][2]
        azimuth = 180
        num_modules = my_df['num_cells'].sum()

        ################################################################################

        #_____________________________Array Parameters_____________________________#
        ## modules per string
        mod_per_str = 1
        parallel_str = 1
        res = runSimulation(weather_url, module_name, inverter_name, tilt, azimuth,
                        mod_wgt, g, air_density, h_high, lifetime, degr_rate_percent,
                        mism_loss, diode_conn_loss, dc_wiring_loss, track_loss,
                        name_plate_loss, dc_optim_loss, snow_loss, mod_per_str,
                        parallel_str)

        (annual_dc_energy, monthly_dc_energy, daily_dc_energy,
        lifetime_dc_energy, surf_tilt, v_mp) = res

        annual_dc_energy_opt = annual_dc_energy*num_modules
        monthly_dc_energy_opt = monthly_dc_energy*num_modules
        daily_dc_energy_opt= daily_dc_energy*num_modules


        ##############
        annual_dc_energy_df['Optimized Tilt'] = annual_dc_energy_opt
        annual_dc_energy_df.to_excel(writer2,
                                 index=False,
                                 sheet_name=file_path[0:-14])


        #-----------------AZIMUTH OPTIMIZATION--------------------------------------#
        face0_starting_azim = np.round(my_df.azim_angles[0],2)
        loop_count = 0
        ref_face_azim = face0_starting_azim

        azim_energy_res = {}

        detailed_res = {}

        while True:

            single_azim_res = {}

            loop_count += 1
            print("iter = " + str(loop_count))

            my_df.azim_angles = my_df.azim_angles.apply(increase_azim,
                                                        args=(azim_step,))
            ref_face_azim = np.round(my_df.azim_angles[0],2)

            print(my_df.azim_angles[0])

            my_dict = my_df.transpose().to_dict('list')
            new_keys = ["Face "+ str(x) for x in my_dict.keys()]
            my_params_list = list(my_dict.values())
            my_building_faces = dict(zip(new_keys, my_params_list))

            annual_dc_energy_temp = 0

            for a_face in my_building_faces:

                tilt = my_building_faces[a_face][0]
                azimuth = my_building_faces[a_face][1]
                num_modules = my_building_faces[a_face][2]

                ########################################################################

                mod_per_str = 1
                parallel_str = 1
                res = runSimulation(weather_url, module_name, inverter_name, tilt,
                                    azimuth, mod_wgt, g, air_density, h_high, lifetime,
                                    degr_rate_percent, mism_loss, diode_conn_loss,
                                    dc_wiring_loss, track_loss, name_plate_loss,
                                    dc_optim_loss, snow_loss, mod_per_str, parallel_str)

                (annual_dc_energy, monthly_dc_energy, daily_dc_energy,
                lifetime_dc_energy, surf_tilt, v_mp) = res

                single_azim_res[a_face] = annual_dc_energy*num_modules

                annual_dc_energy_temp += annual_dc_energy*num_modules

            detailed_res['Azim = ' + str(ref_face_azim)] = single_azim_res

            face0_azim_key = np.round(my_df.azim_angles[0],2)
            azim_energy_res[face0_azim_key] = [annual_dc_energy_temp]


            if face0_azim_key == face0_starting_azim:
                print("Iteration Complete")
                break
            if loop_count == 360:
                print("Iteration step too small")
                break

        detailed_res_df = pd.DataFrame(detailed_res)
        detailed_res_df = detailed_res_df.transpose()
        detailed_res_df.reset_index(inplace=True)
        detailed_res_df.to_excel(writer1,
                                 index=False,
                                 sheet_name=file_path[0:-14])
        
        sim_end = time.time()
        print('Simulation Time for ' + file_path[0:-4] + ': ' + str(np.round(sim_end - sim_start,2)))

    writer1.close()
    writer2.close()
    

my_long_sim_code()

---------------Simulating inukshuk-sam-input in London--------------
iter = 1
351.99986072878323
iter = 2
1.9998607287832328
iter = 3
11.999860728783233
iter = 4
21.999860728783233
iter = 5
31.999860728783233
iter = 6
41.99986072878323
iter = 7
51.99986072878323
iter = 8
61.99986072878323
iter = 9
71.99986072878323
iter = 10
81.99986072878323
iter = 11
91.99986072878323
iter = 12
101.99986072878323
iter = 13
111.99986072878323
iter = 14
121.99986072878323
iter = 15
131.99986072878323
iter = 16
141.99986072878323
iter = 17
151.99986072878323
iter = 18
161.99986072878323
iter = 19
171.99986072878323
iter = 20
181.99986072878323
iter = 21
191.99986072878323
iter = 22
201.99986072878323
iter = 23
211.99986072878323
iter = 24
221.99986072878323
iter = 25
231.99986072878323
iter = 26
241.99986072878323
iter = 27
251.99986072878323
iter = 28
261.99986072878323
iter = 29
271.99986072878323
iter = 30
281.99986072878323
iter = 31
291.99986072878323
iter = 32
301.99986072878323
iter = 33
311.9998

In [ ]:
# building_model_list = ["output"]

# writer = pd.ExcelWriter('azimuth_opt_results.xlsx')
# detailed_res_df = pd.DataFrame(detailed_res)
# detailed_res_df = detailed_res_df.transpose()

# for model_num in range(len(building_model_list)):
#     detailed_res_df.reset_index(inplace=True)
#     detailed_res_df.to_excel(writer,
#                              index=False,
#                              sheet_name=building_model_list[model_num])

# writer.close()

# # pd.DataFrame.to_csv(load, hp_model_list[model_num]+".csv")

In [ ]:
# pip install squarify

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import squarify
# plt.figure(figsize=(10,10))
# # Sample data
# values = [250, 120, 280, 320, 140, 95, 25]

# # Treemap
# squarify.plot(sizes = plot_values,
#               #label=['a'+ str(i) for i in range(80)],
#               color=sns.color_palette("crest", len(plot_values))
# )

# # Remove the axis:
# plt.axis("off")
# plt.legend()
# # plt.show()

## Simulation Core Code

In [ ]:
# module_database_url = 'CEC_Modules.csv'
# inverter_database_url = 'CEC_Inverters.csv'

# simulation_data = ['London Ontario':['/content/london_ontario_42.984267_-81.247534_psm3-tmy_60_tmy.csv']]
# weather_url = '/content/south_africa_msg-iodc_60_2019.csv'
# module_name = 'SunPower SPR-X21-335'
# inverter_name = 'SolarEdge Technologies Ltd : SE6000x [208V]'
# azimuth = 0
# tilt = 37
# mod_wgt = 20
# g = 10
# air_density = 1.225
# h_high = 1.04+0.15
# lifetime = 26
# degr_rate_percent = 0.5

# #___________________________________DC Losses__________________________________#
# ## mismatch losses in percentage
# mism_loss = 2
# diode_conn_loss = 0.5
# dc_wiring_loss = 2
# track_loss = 0
# name_plate_loss = 0
# dc_optim_loss = 0
# snow_loss = 0
# #_______________________________Array Parameters_______________________________#
# ## modules per string
# mod_per_str = 6
# parallel_str = 3
# res = runSimulation(weather_url, module_name, inverter_name, tilt, azimuth,
#                   mod_wgt, g, air_density, h_high, lifetime, degr_rate_percent,
#                   mism_loss, diode_conn_loss, dc_wiring_loss, track_loss,
#                   name_plate_loss, dc_optim_loss, snow_loss, mod_per_str,
#                   parallel_str)

# (annual_ac_energy, monthly_ac_energy, daily_ac_energy,
#  lifetime_ac_energy, surf_tilt, v_mp) = res

In [ ]:
# module_database_url = 'CEC_Modules.csv'
# inverter_database_url = 'CEC_Inverters.csv'

# module_name = 'SunPower SPR-X21-335'
# inverter_name = 'SolarEdge Technologies Ltd : SE6000x [208V]'

# city_name = 'London_Ontario'
# weather_url = '/content/london_ontario_42.984267_-81.247534_psm3-tmy_60_tmy.csv'



# annual_ac_energy_dic = {}
# monthly_ac_energy_dic = {}
# daily_ac_energy_dic = {}




# tilt = 34
# azimuth = 180
# # num_modules = 231+222+155+108+526+342+254+143+233+296+452+158+215+143+144+119
# ############################################################################
# mod_wgt = 20
# g = 10
# air_density = 1.225
# h_high = 1.04+0.15
# ############################################################################

# lifetime = 26
# degr_rate_percent = 0.5

# #________________________________DC Losses_________________________________#
# ## mismatch losses in percentage
# mism_loss = 2
# diode_conn_loss = 0.5
# dc_wiring_loss = 2
# track_loss = 0
# name_plate_loss = 0
# dc_optim_loss = 0
# snow_loss = 0
# #_____________________________Array Parameters_____________________________#
# ## modules per string
# mod_per_str = 6
# parallel_str = 3
# res = runSimulation(weather_url, module_name, inverter_name, tilt, azimuth,
#                 mod_wgt, g, air_density, h_high, lifetime, degr_rate_percent,
#                 mism_loss, diode_conn_loss, dc_wiring_loss, track_loss,
#                 name_plate_loss, dc_optim_loss, snow_loss, mod_per_str,
#                 parallel_str)

# (annual_dc_energy, monthly_dc_energy, daily_dc_energy,
# lifetime_dc_energy, surf_tilt, v_mp) = res
# annual_dc_energy = annual_dc_energy
# # annual_ac_energy_dic[a_face] = [annual_ac_energy]
# # monthly_ac_energy_dic[a_face] = monthly_ac_energy
# # daily_ac_energy_dic[a_face]= daily_ac_energy

# # annual_ac_energy_df = pd.DataFrame.from_dict(annual_ac_energy_dic)
# # monthly_ac_energy_df = pd.DataFrame.from_dict(monthly_ac_energy_dic)
# # daily_ac_energy_df = pd.DataFrame.from_dict(daily_ac_energy_dic)

In [ ]:
# my_df.transpose().sum(axis=1)

In [ ]:
# azim_energy_res

In [ ]:
# my_df.azim_angles = my_df.azim_angles.apply(increase_azim, args=(azim_step,))
# my_df

In [ ]:
# 365-360

In [ ]:
file_path[0:-4]

In [ ]:
x = input('Name your building')

print(os.path.exists(x))